## `SciPy.fftpack` - Rychlá Fourierova transformace
`scipy.fftpack` je modul, který poskytuje nástroje pro rychlé Fourierovy transformace (FFT) a jejich inverzní operace. FFT je algoritmus, který umožňuje rychle vypočítat diskrétní Fourierovu transformaci (DFT) a její inverzní. DFT je matematický nástroj používaný k analýze a zpracování signálů, zejména pro rozklad signálů na frekvenční spektrum.

In [ ]:
import numpy as np
from scipy import fftpack
from IPython.display import Audio
import matplotlib.pyplot as plt


In [ ]:
# Nastavení vzorkovací frekvence pro zvukový signál
vzorkovaci_frekvence = 8000
# Délka signálu
doba_trvani = 1

# Vytvoříme časovou osu
t = np.linspace(0, doba_trvani, int(doba_trvani * vzorkovaci_frekvence))

# Definujeme základní frekvenci a harmonické
frekvence = 800
harmonicka2 = 2 * frekvence
harmonicka3 = 3 * frekvence

# Vytvoříme signál s harmonickými frekvencemi
signal = (np.sin(2 * np.pi * frekvence * t) +
          0.5 * np.sin(2 * np.pi * harmonicka2 * t) +
          0.3 * np.sin(2 * np.pi * harmonicka3 * t))

# Přidáme šum do signálu
zasumeny_signal = signal + 0.5 * np.random.randn(len(signal))

In [ ]:
# Čistý signál
# Použijte IPython.display.Audio pro přehrání signálu
Audio(data=signal, rate=vzorkovaci_frekvence)

In [ ]:
# zašuměný signál
# přehrajeme signál
Audio(data=zasumeny_signal, rate=vzorkovaci_frekvence)

In [ ]:
# Aplikujeme rychlou Fourierovu transformaci (FFT)
noisy_fft = fftpack.fft(zasumeny_signal)
clean_fft = fftpack.fft(signal)

# Vypočítáme frekvenční spektrum
frequencies = fftpack.fftfreq(len(signal), doba_trvani / vzorkovaci_frekvence)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))

ax1.plot(t, signal)
ax1.set_title('Časový průběh signálu')
ax1.set_xlabel('Čas [s]')
ax1.set_ylabel('Amplituda')
ax1.set_xlim(0, 0.01)

ax2.plot(frequencies, np.abs(clean_fft))
ax2.set_title('Frekvenční spektrum')
ax2.set_xlabel('Frekvence [Hz]')
ax2.set_ylabel('Amplituda')

plt.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))

ax1.plot(t, zasumeny_signal)
ax1.set_title('Časový průběh signálu')
ax1.set_xlabel('Čas [s]')
ax1.set_ylabel('Amplituda')
ax1.set_xlim(0, 0.01)

ax2.plot(frequencies, np.abs(noisy_fft))
ax2.set_title('Frekvenční spektrum')
ax2.set_xlabel('Frekvence [Hz]')
ax2.set_ylabel('Amplituda')

plt.tight_layout()
plt.show()


In [ ]:
# Vidíme, že šum je ve frekvenčním spektru pěkně oddělen od základní frekvence

# Filtrujeme šum ve frekvenčním spektru tak, že vypustíme všechny malé amplitudy
maska = np.abs(noisy_fft) > 200
filtrovana_fft = noisy_fft * maska

# Aplikujeme inverzní rychlou Fourierovu transformaci (IFFT)
filtrovany_signal = np.real(fftpack.ifft(filtrovana_fft))

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))

ax1.plot(t, filtrovany_signal)
ax1.set_title('Časový průběh signálu')
ax1.set_xlabel('Čas [s]')
ax1.set_ylabel('Amplituda')
ax1.set_xlim(0, 0.01)

ax2.plot(frequencies, np.abs(filtrovana_fft))
ax2.set_title('Frekvenční spektrum')
ax2.set_xlabel('Frekvence [Hz]')
ax2.set_ylabel('Amplituda')
ax2.set_xlim(0, 2500)

plt.tight_layout()
plt.show()


In [ ]:
# filtrovaný signál
# přehrajeme signál
Audio(data=filtrovany_signal, rate=vzorkovaci_frekvence)
